In [2]:
!pip install deeplabcut[tf,modelzoo]


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 KB 27.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 39.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 35.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=493a9ae358fc194b0aa087d66d9f1c2114baae349a5c52d4be2131b543fb93f8
  Stored in directory: /uoa/home/t08io22/.cache/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy
You should consider upgrading via the '/opt/software/uoa/spack-sw/linux-rhel8-x86_64/gcc-12.1.0/python-3.9.12-u2o5ndnqcbr26nc573ubjrvv7o5n5wts/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import deeplabcut as dlc
import tensorflow
import os
from pathlib import Path
from datetime import date
import pandas as pd

video_path = Path(os.getcwd() + "/dummy-video.mp4")

# Ensure video file exists
assert video_path.is_file()


2023-06-12 10:13:48.950202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 10:13:49.526630: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 10:13:54.010813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/software/uoa/spack-sw/linux-rhel8-x86_64/gcc-12.1.0/libiconv-1.16-o5xic5hftcncnk3zgrajfx6nbjnaenun/lib:/opt/software/uoa/spack-sw/linux-rhel8-x86_64/gcc-12.1.0/r-4.1.3-gixu37cbx7dcnwzya2lu7jimi4qd2gl

Loading DLC 2.3.5...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Load weights from pre-trained macaque model.

In [ ]:
config_path, train_config_path = dlc.create_pretrained_project(
    "mrp",
    "scorer",
    [video_path],
    videotype="mp4",
    model="full_macaque",
    analyzevideo=False,         # True: a labeled video is created, else only weights downloaded
    createlabeledvideo=True,    # ? no documentation available
    copy_videos=True,           # From Colab: must leave copy_videos=True (?)
)
config_file = dlc.auxiliaryfunctions.read_plainconfig(config_path)
# Set iteration to value +1 from init_weights (possibly later than here?)
config_file["iteration"] = 1

Use command line functionality to load COCO annotation and image data into required labeled-data sub-directory.

In [60]:
today = date.today()
data_dir = Path(os.getcwd() + f"/mrp-scorer-{today}/labeled-data/dummy-video")
print(data_dir)

# Load annotations if desired file exists
anno_file = Path("coco2017_val_subset_anno.h5")
assert anno_file.is_file()
!cp {str(anno_file)} {data_dir / "CollectedData_scorer.h5"}


/uoa/scratch/users/t08io22/pommes-test/mrp-scorer-2023-06-09/labeled-data/dummy-video


Extract list of files that are actually to copy, and use the list to copy image files into required folder.

Load extract indices for train/test from full dataframe that we load here, but manipulate during preprocessing already.

(Note: # (If copying the whole folder: !cp ../coco/val2017/* {data_dir}))

In [61]:
df = pd.read_hdf(data_dir / "CollectedData_scorer.h5")

# Extract train/test information

df_train = df[df["istrain", "", ""] == True]
is_train = df["istrain", "", ""].to_list()
train_indices = [index for index, flag in enumerate(is_train) if flag]

df_val = df[df["istrain", "", ""] == False]
is_val = [not flag for flag in is_train]
val_indices = [index for index, flag in enumerate(is_val) if flag]

# Drop flag column

df = df.drop(columns=["istrain"], axis=1, level=0)
df.to_hdf(data_dir / "CollectedData_scorer.h5", key="df", mode="w")

# Copy trains set images

file_paths = df_train.index.to_list()
dataset_dir = Path("../coco/train2017")
with open("files-to-copy_train.txt", "w") as outfile:
    outfile.writelines(map(lambda path: str(dataset_dir / os.path.basename(path)) + "\n", file_paths))

!cat files-to-copy_train.txt | xargs -I % cp % {data_dir}

# Repeat for validation sets

file_paths = df_val.index.to_list() # TODO: Change to df when still in test phase (!)
dataset_dir = Path("../coco/val2017")
with open("files-to-copy_val.txt", "w") as outfile:
    outfile.writelines(map(lambda path: str(dataset_dir / os.path.basename(path)) + "\n", file_paths))

!cat files-to-copy_val.txt | xargs -I % cp % {data_dir}


cp: cannot stat '../coco/train2017/000000142620.jpg': No such file or directory
cp: cannot stat '../coco/train2017/000000163682.jpg': No such file or directory
cp: cannot stat '../coco/train2017/000000209753.jpg': No such file or directory
cp: cannot stat '../coco/train2017/000000275058.jpg': No such file or directory


Assert COCO training data has been loaded into required folder. When run for the first time, this also plots the keypoint annotations with the frames.

In [62]:
dlc.check_labels(config_path)


Creating images with labels by scorer.


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 13.64it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


Here I am unsure: Do I need to run create_training_dataset? I would like to keep the splits as defined by COCO, because this prevents bias introduction. However, as for data, this step will create a .mat file, which contains the
address of the images as well as the target postures, and a .pickle file, which contains the meta
information about the training dataset - DLC probably requires this.

So the solution I think I will go with is to use create_training_dataset and create a training dataset with default settings, grab that .h5, and figure out the indices of the images I want in training/testing. Using my preprocessing routine, I merge the train/test DataFrames, sort them lexicographically by integer (as DLC does when splitting), and mark which images to be used for training. After doing that, I know which indices to enter here and I can use the DLC function.

#### Think:
Do we want shuffles? I can then create two model instances that have the identical training set, allowing me to assess the role of various parameters on the performance of DLC (each shuffle would include all of the same images, just allocated differently - unsure if this means just changing the order or also the set to which each is assigned...). Also, I am working with this multiple times on the same day, I might need to use shuffles because DLC might not overwrite certain files/folders.

/!\ After throwing out images with too few keypoints, does the split ratio still make sense?

/!\ Consider setting the augmenter type to the one used during pre-training (no information found in journal article.

In [63]:
dlc.create_training_dataset(
    config=config_path,
    trainIndices=[train_indices,],
    testIndices=[val_indices,],
    net_type="resnet_50", # set to the same neural network as pre-trained model
)


You passed a split with the following fraction: 80%
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.8, 1, (array([0, 1, 2, 3]), array([4])))]

Actual training can start now.

Should I set `keepdeconvweights` to `False`? I think so. This means I won't update their weights during training and assumes that the pre-trained features are already relevant (learned meaningful features from the macaques) and can generalize well. (It would definitely be necessary if I used a different set or different names of bodyparts.)

#### Think:

/!\ Do I have to set "trainingsetindex"? (Integer specifying which training set fraction to use. By default it is the first index value listed, note that Training Fraction is a list in config.yaml)

/!\ DLC apparently doesn't update the TrainingFraction in config.yaml automatically. Is that a problem?

How can we optimize the model (or its parameters)? Is it through shuffling? Potentially check out `create_training_model_comparison`.


In [58]:
# unsure if this is the right iteration value...
pose_config_path = Path(os.getcwd()) / f"/mrp-scorer-{today}/dlc-models/iteration-1/mrp{today.strftime('%b%e').replace(' ','')}-trainset{str(int(config_file['TrainingFraction'][0] * 100))}shuffle1/train/pose_cfg.yaml"
print(pose_config_path)
assert pose_config_path.is_file()
pose_config_file = dlc.auxiliaryfunctions.read_plainconfig(pose_config_path)

# In pose_cfg.yaml of the latest iteration, change init_weights to the last snapshot of the pre-trained model
init_weights_path = Path(os.getcwd()) / f"/mrp-scorer-{today}/dlc-models/iteration-0/mrp{today.strftime('%b%e').replace(' ','')}-trainset{str(int(config_file['TrainingFraction'][0] * 100))}shuffle1/train/snapshot-1030000"
pose_config_file["init_weights"] = str(init_weights_path)


/mrp-scorer-2023-06-09/dlc-models/iteration-1/mrpJun9-trainset95shuffle1/train/pose_cfg.yaml


AssertionError: 

In [2]:
dlc.train_network(
    config_path,
    keepdeconvweights=False,
)
# /!\ before GPU access: set max_iters


NameError: name 'config_path' is not defined

==================================================================================================================

Now, we can evaluate the performance using the mean average Euclidean error (MAE; which is proportional to the average root mean square error) between the manual labels and the ones predicted. Hopefully, DLC automatically uses the correct test split. Results are stored as .csv file in a subdirectory under evaluation-results, displayed for all pairs and only likely pairs (>p-cutoff). (This helps to exclude, for example, occluded body parts.)

In [ ]:
dlc.evaluate_network(config_path, plotting=True) # setting plotting to True plots all the testing and training frames with the manual and predicted labels

DLC offers much more: here goes the optional stuff included in demos I read.

In [ ]:
# Optional (I guess?) Don't know if it works without the "true" videos
dlc.analyze_videos(path_to_config, [video_path])

# Optional (I guess?) Create labeled video, watch options
dlc.create_labeled_video(path_to_config, [video_path])

# Optional (I guess?) Plot trajectories of all body parts across entire video
dlc.plot_trajectories(path_to_config, [video_path])

# Optional: Extract outliers and refine labels
dlc.extract_outlier_frames(path_to_config, [video_path])
dlc.refine_labels(path_to_config)
#(deeplabcut.label_frames(path_config_file), deeplabcut.check_labels(path_config_file))
dlc.create_training_dataset(path_to_config) #?

# => reiterate process, mind merge_datasets if labels refined!